In [17]:
import requests
import pandas as pd

In [10]:
# -------------- CONFIG --------------
CLIENT_ID = "632f59ffb300474e8c2696f44a359090"
CLIENT_SECRET = "RbK5nlZBW3lKXv40W5e7WkKRp24RLIjh"
REGION = "eu"

In [11]:
REALM_SLUG = "soulseeker"
CHAR_NAME = "Sphere"
NAMESPACE = f"profile-classic1x-{REGION}"

In [18]:
# ------------------------------------
def get_access_token():
    url = f"https://eu.battle.net/oauth/token"
    resp = requests.post(url, data={"grant_type": "client_credentials"},
                         auth=(CLIENT_ID, CLIENT_SECRET))
    resp.raise_for_status()
    return resp.json()["access_token"]

def get_character_gear(token, character_name):
    url = (
        f"https://{REGION}.api.blizzard.com/profile/wow/character/"
        f"{REALM_SLUG}/{character_name.lower()}/equipment"
    )
    params = {"namespace": NAMESPACE, "locale": "en_GB"}
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 404:
        print(f"Character '{character_name}' not found.")
        return []

    response.raise_for_status()
    gear_data = response.json()

    gear_list = []
    for item in gear_data.get("equipped_items", []):
        gear_list.append({
            "Name": character_name,
            "Slot": item["slot"]["name"],
            "Item": item["name"]
        })

    return gear_list

def get_raid_gear_dataframe(raider_names):
    token = get_access_token()
    all_gear = []

    for name in raider_names:
        try:
            gear = get_character_gear(token, name)
            all_gear.extend(gear)
        except Exception as e:
            print(f"Error retrieving gear for {name}: {e}")

    return pd.DataFrame(all_gear)


In [26]:
# Example usage:
raiders = ["Sphere", "Oxigodx", "Amozuggerz","Brezhyisleep","Fiorin","Qxisbkbk","Zeyv","Cptn","Eleenbk","Thotiana","Vástago","Lastzug","Ethor","Ibbernezer", "Loparn"]
df_gear = get_raid_gear_dataframe(raiders)

In [27]:
df_gear.head()

,Name,Slot,Item
0,Sphere,Head,Lionheart Helm
1,Sphere,Neck,Onyxia Tooth Pendant
2,Sphere,Shoulders,Bloodsoaked Pauldrons
3,Sphere,Chest,Cadaverous Armor
4,Sphere,Waist,Onslaught Girdle


In [28]:
df_gear.pivot(index='Name',columns='Slot',values='Item')

Slot,Back,Chest,Feet,Hands,Head,Legs,Main Hand,Neck,Off Hand,Ranged,Ring 1,Ring 2,Shirt,Shoulders,Tabard,Trinket 1,Trinket 2,Waist,Wrist
Name,,,,,,,,,,,,,,,,,,,
Amozuggerz,Cloak of Draconic Might,Savage Gladiator Chain,Boots of Heroism,Sacrificial Gauntlets,Lionheart Helm,Eldritch Reinforced Legplates,Empyrean Demolisher,Onyxia Tooth Pendant,Mirah's Song,Riphook,Master Dragonslayer's Ring,Quick Strike Ring,Formal White Shirt,Spaulders of Heroism,NaN,Hand of Justice,Blackhand's Breadth,Onslaught Girdle,Zandalar Vindicator's Armguards
Brezhyisleep,Stoneskin Gargoyle Cape,Cadaverous Armor,Bloodmail Boots,Sacrificial Gauntlets,Lionheart Helm,Dark Heart Pants,Empyrean Demolisher,Onyxia Tooth Pendant,Brutality Blade,Skull Splitting Crossbow,Quick Strike Ring,Master Dragonslayer's Ring,NaN,Truestrike Shoulders,Guild Tabard,Blackhand's Breadth,Hand of Justice,Onslaught Girdle,Zandalar Vindicator's Armguards
Cptn,Puissant Cape,Savage Gladiator Chain,Battlechaser's Greaves,Edgemaster's Handguards,Lionheart Helm,Cloudkeeper Legplates,"Crul'shorukh, Edge of Chaos",The Eye of Hakkar,Deathbringer,Archstrike Bow of the Tiger,Painweaver Band,Master Dragonslayer's Ring,NaN,Drake Talon Pauldrons,Guild Tabard,Hand of Justice,Diamond Flask,Onslaught Girdle,Zandalar Vindicator's Armguards
Eleenbk,Cape of the Black Baron,Zandalar Vindicator's Breastplate,Boots of Heroism,Sacrificial Gauntlets,Lionheart Helm,Abyssal Plate Legplates of Striking,Nightmare Blade,Amulet of the Darkmoon,Mirah's Song,Satyr's Bow,Painweaver Band,Magni's Will,NaN,Spaulders of Heroism,NaN,Diamond Flask,Hand of Justice,Onslaught Girdle,Zandalar Vindicator's Armguards
Ethor,Stoneskin Gargoyle Cape,Breastplate of Valor,Boots of Heroism,Gauntlets of Heroism,Lionheart Helm,Eldritch Reinforced Legplates,Warblade of the Hakkari,Amulet of the Darkmoon,Blackguard,Satyr's Bow,Blackstone Ring,Magni's Will,Formal White Shirt,Spaulders of Heroism,NaN,Diamond Flask,Hand of Justice,Brigam Girdle,Vambraces of the Sadist
Fiorin,Might of the Tribe,Deathdealer Breastplate,Sapphiron's Scale Boots,Sacrificial Gauntlets,Lionheart Helm,Abyssal Mail Legguards of Striking,Chromatically Tempered Sword,Onyxia Tooth Pendant,Mirah's Song,Skull Splitting Crossbow,Master Dragonslayer's Ring,Blackstone Ring,Formal White Shirt,Spaulders of Valor,Guild Tabard,Blackhand's Breadth,Hand of Justice,Zandalar Vindicator's Belt,Zandalar Vindicator's Armguards
Ibbernezer,Cape of the Black Baron,Savage Gladiator Chain,Sapphiron's Scale Boots,Devilsaur Gauntlets,Lionheart Helm,Devilsaur Leggings,Nightmare Blade,The Eye of Hakkar,Blackguard,Blastershot Launcher,Master Dragonslayer's Ring,Seal of the Gurubashi Berserker,Stylish Red Shirt,Bloodsoaked Pauldrons,NaN,Nifty Stopwatch,Blackhand's Breadth,Onslaught Girdle,Zandalar Vindicator's Armguards
Lastzug,Cape of the Black Baron,Cadaverous Armor,Bloodmail Boots,Devilsaur Gauntlets,Lionheart Helm,Devilsaur Leggings,Brutality Blade,The Eye of Hakkar,Mirah's Song,Riphook,Master Dragonslayer's Ring,Seal of the Gurubashi Berserker,Red Linen Shirt,Bloodsoaked Pauldrons,NaN,Hand of Justice,Blackhand's Breadth,Zandalar Vindicator's Belt,Vambraces of the Sadist
Loparn,Eskhandar's Pelt,Zandalar Vindicator's Breastplate,Boots of Heroism,Gauntlets of Heroism,Lionheart Helm,Bloodsoaked Legplates,Bloodlord's Defender,Beads of Ogre Might,Blackguard,Satyr's Bow,Magni's Will,Blackstone Ring,Red Linen Shirt,Spaulders of Heroism,Guild Tabard,Blackhand's Breadth,Hand of Justice,Zandalar Vindicator's Belt,Zandalar Vindicator's Armguards
